<a href="https://colab.research.google.com/github/Covpet/Covpet/blob/main/ADS2_(2022B)_Tutorial_2_PySpark_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ADS2 - Tutorial 2 - PySpark Basics
Learning Outcomes:
*   Importing data into Spark dataframes from .csv files
*   Exploring and manipulating data tables with Spark SQL
*   Write data to a file

**Methods and Functions:**


```
spark
    .read
    .sql

dataframe
    .show()
    .printSchema
```



To begin, colab doesn't come with PySpark available by default, you will need to run the filling blocks of code to install it.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Apache Spark uses Java, so first we must install that
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# Unpack Spark from google drive
!tar xzf /content/drive/MyDrive/spark-3.3.0-bin-hadoop3.tgz

In [ ]:
# Set up environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.3.0-bin-hadoop3"

In [ ]:
# Install findspark, which helps python locate the psyspark module files
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
# Finally, we initialse a "SparkSession", which handles the computations
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

# Exercise 1

for this tutorial, you will explore a dataset of house price data from California. The .csv for this data is available on canvas, as well as from [Kaggle](https://www.kaggle.com/camnugent/california-housing-prices).

 * longitude
 * latitude
 * housing_median_age
 * total_rooms
 * total_bedrooms
 * population
 * households
 * median_income
 * median_house_value
 * ocean_proximity

Download the data, then upload it in the files panel on the left of the colab window. You can copy the path to this file by right-clicking it after it's uploaded.

Begin by loading the dataset into a Spark DataFrame. Certain options can to be set using the `.read.option(key, value)` methods. A full list of the options for .CSV files can be found here: [CSV Files](https://spark.apache.org/docs/latest/sql-data-sources-csv.html). Set the sperator option to commas, and the header option to True.

Finally, you need to tell the Reader where the .csv file is.

When the file is loaded, show the first 5 rows of the data and print the schema.

In [ ]:
### Load the California Housing Prices Dataset
# .read, .option, .csv
csvpath = '/content/housing.csv'

housingdf= spark.read.csv(csvpath, header=True)
#print(housingdf.schema)

# Show the first 5 rows of data and print the schema
show_rows= 5

### Show the first 5 rows of data and print the schema
# .show, .printSchema
housingdf.show(show_rows)
housingdf.printSchema()



+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

The Schema is the database blueprint which specifies the column name, data type, whether the field is nullable, and any extra metadata. In PySpark, these are StructTypes and StructFields. You should have found that the DataFrame you loaded has only strings as the datatypes. This isn't useful for numerical data. fortunately, there are a number of ways to set the schema of a DataFrame when you load it.

The first, and simplest way, is to set the `inferSchema` option to `True` in the read call. Reload the DataFrame with this option set, and preint the schema.

In [ ]:
### Load the DataFrame again, this time with the inferSchema option enabled
housingDf= spark.read.csv(csvpath, header=True, inferSchema=True)
show_rows= 5

housingDf.show(show_rows)
housingDf.printSchema()




+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

For very large tables, inferring the schema can be computationally costly, as PySpark must run an additional pass over the dataset. Instead, you can set predefined schema. One way to do this is by defining a StructType, with a list individual StructFields for each column.

```
schema = StructType([StructField_1, StructField_2, ...])
```

The first StructField is provided below, complete the list for all the columns in the dataset.

Reload the DataFrame, this time replacing the inferSchema `.option()` call, with `.schema(userDefinedSchema)`. Print the new schema and check that it is correct

In [ ]:
### Load the DataFrame using a schema define with StructType and StructField
from pyspark.sql.types import DoubleType, StringType, StructType, StructField

# Complete the schema
schema = StructType([
    StructField('longitude', DoubleType(), True),
    StructField('latitude', DoubleType(), True),
    StructField('housing_median_age', DoubleType(), True),
    StructField('total_rooms', DoubleType(), True),
    StructField('total_bedrooms', DoubleType(), True),
    StructField('population', DoubleType(), True),
    StructField('households', DoubleType(), True),
    StructField('median_income', DoubleType(), True),
    StructField('median_house_value', DoubleType(), True),
    StructField('ocean_proximity', StringType(), True)
])

### Reload the DataFrame with the new schema, then printSchema to check it
housingDf= spark.read.csv(csvpath, header=True, schema=schema, sep=',')
show_rows= 5

housingDf.show(show_rows)
housingDf.printSchema()

# .schema
# Load .csv with header, user defined schema and ',' seperators
# Show the first 5 rows of data and print the schema


+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

Finally, you can define the schema with a DDL (Data Definition Language) string. In this case, the string defines each column name and data type pair, and can be fed into the same `.schema()` method as before. Try this now, and print the schema.

In [ ]:
### Load the DataFrame using a DLL string formatted schema
DDLSchema = "longitude double, latitude double, housing_median_age double, total_rooms double, total_bedrooms double, population double, households double, median_income double, median_house_value double, ocean_proximity string"
### Reload the DataFrame with the new schema, then printSchema to check it
housingDf= spark.read.csv(csvpath, header=True, schema=DDLSchema)
show_rows= 5

housingDf.show(show_rows)
housingDf.printSchema()


+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

# Exercise 2

To save data stored in a DataFrame, use the `.write` method. you can save your data to a number of formats with PySpark. In addition to saving the data as a new .csv, popular formats include [Parquet files](https://parquet.apache.org/documentation/latest/) and JSON files.

Save your DataFrame as a .csv, .parquet, and .JSON file. For the parquet file, set the `.option()` `'compression', 'snappy'`. For the csv file, set the `'header', 'True'`, and `'delimeter', ','`.

In [ ]:
### Save the DataFrame as a .csv, .parquet and .JSON
# .write, .option, .csv, .parquet, .json
csvpath= '/content/housing2.csv'
parquetpath= '/content/housing.parquet'
jsonpath= '/content/housing.json'

housingDf.write.option('header', 'True').option('sep', ',').csv(csvpath)
housingDf.write.option('compression', 'snappy').parquet(parquetpath)
housingDf.write.json(jsonpath)


An alternative way to save the DataFrame is to specify the `.format()` of the file and use the `.save()` method. Repeat the previous oiperations, but this time use the `.format('string')` method, the `.mode('overwrite')` method, and the `.save('/path/to/file') method.

In [ ]:
### Use the .save() method to save the DataFrame as a .csv, .parquet and .JSON
# .write, .format, .option, .mode, .save

housingDf.write.format('csv').option('header', 'True').option('sep', ',').mode('overwrite').save(csvpath)
housingDf.write.format('parquet').option('compression', 'snappy').mode('overwrite').save(parquetpath)
housingDf.write.format('json').mode('overwrite').save(jsonpath)


# Exercise 3

DataFrames can be manipulated using the built-in SQL API. The methods can be used to select columns from the DataFrame, apply filters and masks, sort, or group data, and much more. In this exercise, you will need the following methods:



```
.select() # one or more column names
.where() # boolean expression
.groupBy() # column name
.count()
.orderBy() # column name, ascending=True/False
```

The SQL operations aren't evaluated immediately, and return a new dataframe. By appending `.show()`, you can trigger the calculation and display the new dataframe.

In [ ]:
### Example: Select the median income and house value columns, sort by income
# .select, .orderBy/.sort

housingDf.select('median_income', 'median_house_value').orderBy('median_income').show(5)
housingDf.select('median_income', 'median_house_value').sort('median_income').show(5)



+-------------+------------------+
|median_income|median_house_value|
+-------------+------------------+
|       0.4999|          500001.0|
|       0.4999|           55000.0|
|       0.4999|           90600.0|
|       0.4999|           73500.0|
|       0.4999|          112500.0|
+-------------+------------------+
only showing top 5 rows

+-------------+------------------+
|median_income|median_house_value|
+-------------+------------------+
|       0.4999|          500001.0|
|       0.4999|           55000.0|
|       0.4999|           90600.0|
|       0.4999|           73500.0|
|       0.4999|          112500.0|
+-------------+------------------+
only showing top 5 rows



In [ ]:
### Select the median house age and house value columns and order by total
### bedrooms in descending order
# .select, .orderBy/.sort

housingDf.select('housing_median_age', 'median_house_value').orderBy('total_bedrooms', ascending=False).show(5)


+------------------+------------------+
|housing_median_age|median_house_value|
+------------------+------------------+
|              11.0|          118800.0|
|               3.0|          153700.0|
|               4.0|          366300.0|
|              21.0|          500001.0|
|               8.0|          253900.0|
+------------------+------------------+
only showing top 5 rows



In [ ]:
### Select median income and house value, where ocean proximity is NEAR_BAY
# .select, .filter/.where
housingDf.select('median_income', 'median_house_value').filter(housingDf.ocean_proximity == "NEAR BAY").show(5)


+-------------+------------------+
|median_income|median_house_value|
+-------------+------------------+
|       8.3252|          452600.0|
|       8.3014|          358500.0|
|       7.2574|          352100.0|
|       5.6431|          341300.0|
|       3.8462|          342200.0|
+-------------+------------------+
only showing top 5 rows



In [ ]:
### Count the number of entries where population > 500
# .filter/.where, .count
housingDf.filter('population > 500').count()



18660

In [ ]:
### Group by ocean proximity and count the number of entries in each category
# .groupBy, .count
housingDf.groupBy('ocean_proximity').count().show()


+---------------+-----+
|ocean_proximity|count|
+---------------+-----+
|         ISLAND|    5|
|     NEAR OCEAN| 2658|
|       NEAR BAY| 2290|
|      <1H OCEAN| 9136|
|         INLAND| 6551|
+---------------+-----+



# Exercise 4

The `Column` class is another way to access and manipulate the data within the DataFrame. You can use Columns to form complex expressions, such as:
```
col('total_bedrooms') / col('total_rooms')
col('median_house_value').desc()
(col('median_income')*1000).cast('int')
```
For the following tasks, use Column objects in the DataFrame transformations. Create a new DataFrame for each task, and show the contents.

In [ ]:
### Example: Create a DataFrame with only rows where population > 500, include
### a column with the number of bedrooms / total number of rooms, and sort by
### descending house value
Newdf= housingDf.filter('population > 500')
# col, .filter/.where, withColumn, .orderBy/.sort, desc
from pyspark.sql.functions import col

# Column expression to calculate ratio of bedrooms and rooms
bed_room_ratio = col('total_bedrooms') / col('total_rooms')
Newdf= Newdf.withColumn('bed_room_ration', bed_room_ratio)
Newdf= Newdf.orderBy('median_house_value', ascending=False)
Newdf.show(5)

# Filter by population, add new column, sort DF


+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|    bed_room_ration|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+
|  -122.22|   37.82|              42.0|     2991.0|         335.0|    1018.0|     335.0|       13.499|          500001.0|       NEAR BAY|0.11200267469073888|
|  -121.93|   37.66|              24.0|     3166.0|         424.0|    1081.0|     400.0|       8.3337|          500001.0|      <1H OCEAN|0.13392293114339862|
|  -122.23|   37.82|              52.0|     3242.0|         366.0|    1001.0|     352.0|      12.2138|          500001.0|       NEAR BAY|0.11289327575570636|
|  -122.25|   37.87|              52.0|      609.0| 

In [ ]:
### Select the population and median house value where
### the median house age is < 20, store the result as a new DataFrame
# col, .select, .where

pop_col= col('population')
med_house_col= col('median_house_value')
housingDf.select(pop_col, med_house_col).where(col('housing_median_age') < 20).show(5)

+----------+------------------+
|population|median_house_value|
+----------+------------------+
|      94.0|           60000.0|
|     762.0|          177500.0|
|     721.0|          108300.0|
|     609.0|          162500.0|
|     546.0|          162500.0|
+----------+------------------+
only showing top 5 rows



In [ ]:
### Create a new DataFrame where the ocean proximity column has been dropped.
# col, .drop
Ocean_col= col('ocean_proximity')
Newdfs= housingdf.drop(Ocean_col).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|
|  -122.25|   37.85|              52.0|     1627.0|         280.0|     565.0|     259.0|       3.8462|          342200.0|
|  -122.25|   37.85|    

In [ ]:
### Create a DataFrame which includes a new column for population per household,
### sort by that column, and rename 'ocean_proximity' to 'location'
# col, .withColumn, .withColumnRenamed, .sort/.orderBy

In [ ]:
### Create a DataFrame with the null values in total_bedrooms removed
# col, .isNotNull, .where/.filter

In [ ]:
### Using the collect method to return the listed values of a column, create a
### scatter plot in matplotlib with 'longitude' vs 'latitude', coloured by the
### 'median_house_value'. Include appropriate axis labels, and a labelled
### colour bar.
# .select, .collect, .scatter, .colorbar
import matplotlib.pyplot as plt

longitude = usersDF.select(col('longitude')).collect()
latitude = []
house_value = []